# Лабораторная 1
## Подраздел: метод Якоби

* Cтудент: Ефимов А.В.
* Группа: М8О-307Б
* Вариант: 7

Загрузка библиотек, настройка лог сообщений и загрузка матрицы:

In [1]:
import math
import numpy as np
import logging, json

MAX_ITER = 1000 # Max iterations in case something goes south

# Configurations
np.set_printoptions(suppress=True)
logging.basicConfig(level=logging.DEBUG)

# Load matrix and eps
with open("task.json", "r") as json_file:
    task = json.load(json_file)

matrix = np.array(task["matrix"])
try:
    eps = task["epsilon"]
except KeyError:
    eps = 10e-5

print(f"Input matrix:\n{matrix}")
print(f"Selected epsilon: {eps}")

Input matrix:
[[-6  6 -8]
 [ 6 -4  9]
 [-8  9 -2]]
Selected epsilon: 0.0001


Метод Якоби основывается на приведении симметрической матрицы к диагональной матрице 
(тогда итоговая матрица будет состоять только из действительных собственных значений) 
за счет ее итеративного умножения на матрицы вращений для зануления недиагональных 
элементов. Матрица считается приведенной, если норма матрицы без диагональных элементов
равна (или близка к) нулю.

Для начала нужно выделить функцию, которая будут искать максимальные элементы вне
диагонали, и функцию для нормы без диагонали:

In [2]:
def non_diagonal_norm(matrix):
    total = 0
    for i in range(len(matrix)):
        for j in range(i + 1, len(matrix[i])):
            total += matrix[i][j] ** 2
    return (math.sqrt(total))

def max_non_diagonal(matrix):
    max_i, max_j = 0, 1
    max_val = abs(matrix[max_i][max_j])
    for i in range(len(matrix)):
        for j in range(i + 1, len(matrix[i])):
            val = abs(matrix[i][j])

            if max_val < val:
                max_val = val
                max_i, max_j = i, j
    return (max_i, max_j)

matr = np.arange(9).reshape((3, 3))
sym_matr = matr @ matr.T
print(sym_matr)
print(non_diagonal_norm(sym_matr))
print(max_non_diagonal(sym_matr))

[[  5  14  23]
 [ 14  50  86]
 [ 23  86 149]]
90.11659114724658
(1, 2)


Следует сделать замечание, что после всех преобразований матрица остается симметричной,
поэтому операции можно делать только над одим из треугольников

Матрица поворота будет высчитываться на основе индексов максимального элемента, поэтому её также можно определить отдельно:

In [3]:
def create_rotation(matrix, i, j):
    rotation = np.identity(len(matrix))
    if matrix[i, i] == matrix[j, j]:
        angle = math.pi / 4
    else:
        angle = math.atan(2 * matrix[i, j] / (matrix[i, i] - matrix[j, j])) / 2

    s = math.sin(angle)
    c = math.cos(angle)

    rotation[i, j] = -s;
    rotation[j, j] = c;
    rotation[j, i] = s;
    rotation[i, i] = c;
    
    return rotation

Итеративная функция Якоби будет иметь вид:

In [4]:
def jacobi_iteration(matrix, eps=10e-5):
    matrix  = matrix.copy()
    vectors = np.identity(len(matrix))
    
    logging.info(f"Input matrix:\n{matrix}")
    logging.info(f"Input eps: {eps}")

    iter_no = 1
    while non_diagonal_norm(matrix) >= eps and iter_no <= MAX_ITER:
        logging.debug(f"Iteration no. {iter_no} -------------------")
        max_i, max_j = max_non_diagonal(matrix)
        rotation = create_rotation(matrix, max_i, max_j)
        
        logging.debug(f"Max element on position {(max_i, max_j)}, "
                      "resulting in rotation:\n{rotation}")

        vectors  = vectors @ rotation

        matrix = matrix @ rotation
        rotation = rotation.T
        matrix = rotation @ matrix
        
        logging.debug(f"Current matrix:\n{matrix}")
        logging.debug(f"Current eigenvectors:\n{vectors.T}")
        
        iter_no += 1
        
    logging.debug(f"Values found in {iter_no} iterations")
    return np.diag(matrix), vectors

In [5]:
values, vectors = jacobi_iteration(matrix)

print("Eigenvalues: ", values)
print("Eigenvectors: ")
print(vectors)

INFO:root:Input matrix:
[[-6  6 -8]
 [ 6 -4  9]
 [-8  9 -2]]
INFO:root:Input eps: 0.0001
DEBUG:root:Iteration no. 1 -------------------
DEBUG:root:Max element on position (1, 2), resulting in rotation:
{rotation}
DEBUG:root:Current matrix:
[[ -6.           9.80613981  -1.95949537]
 [  9.80613981 -12.05538514   0.        ]
 [ -1.95949537   0.           6.05538514]]
DEBUG:root:Current eigenvectors:
[[ 1.          0.          0.        ]
 [ 0.          0.74512802 -0.66692146]
 [ 0.          0.66692146  0.74512802]]
DEBUG:root:Iteration no. 2 -------------------
DEBUG:root:Max element on position (0, 1), resulting in rotation:
{rotation}
DEBUG:root:Current matrix:
[[  1.23521638   0.          -1.57676265]
 [  0.         -19.29060152   1.16337511]
 [ -1.57676265   1.16337511   6.05538514]]
DEBUG:root:Current eigenvectors:
[[ 0.80467792  0.44239114 -0.395959  ]
 [-0.59371159  0.59958806 -0.53665697]
 [ 0.          0.66692146  0.74512802]]
DEBUG:root:Iteration no. 3 -------------------
DEBUG:

Eigenvalues:  [  0.77069675 -19.34414228   6.57344552]
Eigenvectors: 
[[ 0.76204662 -0.59591453 -0.25331961]
 [ 0.62257015  0.56673177  0.53964944]
 [-0.17802067 -0.56894726  0.80287462]]


Проверить можно исходя из ортогональности собственных векторов друг другу:

In [6]:
cnt = len(vectors)

for i in range(cnt):
    for j in range(i + 1, cnt):
        print(f"Vectors {i} and {j}: {np.dot(vectors[:, i], vectors[:, j])}")

Vectors 0 and 1: -1.3877787807814457e-17
Vectors 0 and 2: 2.7755575615628914e-17
Vectors 1 and 2: -5.551115123125783e-17


Видно, что в пределах погрешности они равны нулю.